In [1]:
import json
import pandas as pd
import numpy as np
import a1chemy as a1
import a1chemy.indicators as indicators
import a1chemy.data_source as data_source
from tqdm.notebook import tqdm
import random
import time
import pymongo
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

def magnify():
    width = '80px'
    return [dict(selector="th",
                 props=[('max-width', width), ('width', width), ('word-wrap', 'break-word'), ('text-align', 'center')]),
            dict(selector="td",
                 props=[('max-width', width), ('width', width), ('text-align', 'center')]),
            dict(selector="tr",
                 props=[('max-width', width), ('width', width), ('text-align', 'center')])
]

mongo_client = pymongo.MongoClient("mongodb://localhost:27017/", username='a1chemy', password='1B2C9046-E3CC-447F-9961-E125759BA44F')
mongo_ticks_client = data_source.MongoTicks(mongo_client)
mongo_tags_client = data_source.MongoTags(mongo_client=mongo_client)

# sectors data
# sectors_id = 'tradingview_sectors'
# sectors_id = 'swsindex'
sectors_id = 'csi_sectors'
swsindex_tag_tree = mongo_tags_client.tree(id=sectors_id)
print('swsindex_tag_tree.len({})'.format(len(swsindex_tag_tree.index_map)))
stock_sector_map = {}
sectors = set()


for key, value in swsindex_tag_tree.root.children.items():
    sectors.add(key)
    for id, stock in value.children.items():
        stock_sector_map[stock.values['symbol']] = key
print('stock_sector_map len({})'.format(len(stock_sector_map)))

stocks_tree = mongo_tags_client.tree(id='csi500')
sci_300_sector_map = {}
stocks = []
for k, stock in stocks_tree.root.children.items():
    sector = stock_sector_map[stock.values['symbol']]
    sector_list = sci_300_sector_map.setdefault(sector, [])
    sector_list.append(stock)
    stocks.append(stock.values)


from a1chemy.common.ticks import Ticks
stock_candle_data_map = {}
stock_symbol_list = [stock['symbol'] for stock in stocks]
print(len(stock_symbol_list))
for tick in mongo_ticks_client.find({'symbol':{'$in': stock_symbol_list}}):   
    tick.raw_data['MA20'] = indicators.sma(data=tick.close(), day=20)
    tick.convert_tz(tz='Asia/Tokyo')
    stock_candle_data_map[tick.symbol] = tick

sci_300_sector_count_map = {}
for k, v in sci_300_sector_map.items():
    sci_300_sector_count_map[k] = len(v)

print('\x1b[31m全行业公司\x1b[0m:')
for k, v in sci_300_sector_count_map.items():
    company_name_list = ",".join([stock_candle_data_map[d.values['symbol']].name for d in sci_300_sector_map[k]])
    print("\x1b[31m{}({})\x1b[0m:\n{}".format(k, v, company_name_list))


## 画图
breath_color_list = [
    '#FF0000',
    '#FF3333',
    '#FF6666',
    '#FF9999',
    '#FFCCCC',
    '#CCFFCC',
    '#99FF99',
    '#66FF66',
    '#33FF33',
    '#00CC00'
]

def market_breath_color(v):
    '''
    highlight the maximum in a Series yellow.
    '''
    color_index = 0 if float(v) <= 0 else int(float(v)/10-1)
    return 'background-color: ' + breath_color_list[color_index]


start_time = pd.Timestamp("2020-05-01 00:00:01", tz="Asia/Tokyo")
market_breath = {}
last_day_active_stocks = {}
for k, v in stock_candle_data_map.items():
    for i in range(len(v.index())-100, len(v.index())):
        close = v.close()[i]
        ma20 = v.raw_data['MA20'][i]
        current_day = v.index()[i]
        sector = stock_sector_map[k]
        current_day_data = market_breath.setdefault(str(current_day), {})

        value = current_day_data.setdefault(sector, {})
        all_value = current_day_data.setdefault('all', {})
        above = value.setdefault('above', 0)
        stocks_count = value.setdefault('all', 0)
        
        all_above = all_value.setdefault('above', 0)
        all_stocks_count = all_value.setdefault('all', 0)
        if close > ma20:
            if i == (len(v.index()) -1):
                last_day_sector_active_stock_list = last_day_active_stocks.setdefault(sector, [])
                last_day_sector_active_stock_list.append(v.name)
            current_day_data[sector]['above'] = above + 1
            current_day_data['all']['above'] = all_above + 1
        value['all'] = stocks_count + 1
        all_value['all'] = all_stocks_count + 1

print('----------------------')
print('\x1b[31m最后一天活跃公司\x1b[0m')
for k, v in last_day_active_stocks.items():
    print("\x1b[31m{}\x1b[0m:\n{}".format(k, ",".join(v)))
        
for k, v in market_breath.items():
    for name, value in v.items():
        value['percent'] = round(float(value['above'])*100/float(value['all']),2)

market_breath_show_data = []
for k, v in sorted(list(market_breath.items()), key=lambda x:x[0].lower(), reverse=True):
    result = [k[0:10]]
    for sector in sectors:
        result.append(str(v.get(sector, {}).get('percent', 0)))
    result.append(str(v.get('all',{}).get('percent', 0)))
    market_breath_show_data.append(result)
market_breath_column_name_list = ['日期']
market_breath_column_name_list.extend(sectors)
market_breath_column_name_list.append("ALL")
market_breath_data_frame=pd.DataFrame(market_breath_show_data, columns=market_breath_column_name_list)

result = None
if sectors_id == 'swsindex':
    result = market_breath_data_frame.style.applymap(market_breath_color, subset=market_breath_column_name_list[1:])
else:
    result = market_breath_data_frame.style.applymap(market_breath_color, subset=market_breath_column_name_list[1:]).set_table_styles(magnify())
result

swsindex_tag_tree.len(4564)
stock_sector_map len(4553)
500
全行业公司:
可选消费(57):
东风汽车,歌华有线,海信视像,宇通客车,中体产业,福田汽车,雅戈尔,首旅酒店,中文传媒,海澜之家,江淮汽车,东方明珠,均胜电子,彩虹股份,重庆百货,北汽蓝谷,锦江酒店,四川长虹,王府井,江苏有线,中南传媒,小康股份,拓普集团,郑煤机,际华集团,皖新传媒,凤凰传媒,莱克电气,东方时尚,科沃斯,欧普照明,彤程新材,科博达,飞科电器,华数传媒,鄂武商A,万向钱潮,威孚高科,视觉中国,中鼎股份,宁波华翔,万丰奥威,利欧股份,九阳股份,爱施德,天虹股份,兆驰股份,万里扬,森马服饰,索菲亚,新宝股份,周大生,盈趣科技,蓝色光标,华策影视,捷成股份,光线传媒
公用事业(25):
首创环保,上海电力,华电国际,国网信通,联美控股,申能股份,新奥股份,内蒙华电,贵州燃气,重庆燃气,新天绿能,节能风电,深圳燃气,上海环境,晶科科技,大唐发电,伟明环保,深圳能源,兴蓉环境,中山公用,宝新能源,启迪环境,湖北能源,盈峰环境,碧水源
原材料(90):
山东钢铁,杭钢股份,兴发集团,巨化股份,南山铝业,广晟有色,南钢股份,酒钢宏兴,盛和资源,三友化工,扬农化工,驰宏锌锗,中化国际,方大特钢,方大炭素,山东药玻,厦门钨业,山鹰国际,华谊集团,东阳光,新钢股份,华新水泥,马钢股份,亚泰集团,赤峰黄金,柳钢股份,重庆钢铁,海南橡胶,西部矿业,桐昆股份,北元集团,旗滨集团,金钼股份,海南矿业,金隅集团,和邦生物,新凤鸣,宏和科技,合盛硅业,三美股份,三棵树,南玻A,中金岭南,冀东水泥,晨鸣纸业,安道麦A,攀钢钒钛,铜陵有色,河钢股份,韶钢松山,中色股份,新兴铸管,云铝股份,太钢不锈,鲁西化工,天山股份,云南铜业,鞍钢股份,华菱钢铁,首钢股份,锡业股份,银泰黄金,诚志股份,鸿达兴业,华邦健康,沙钢股份,太阳纸业,中材科技,中泰化学,三钢闽光,湖南黄金,海亮股份,塔牌集团,联化科技,西部建设,齐翔腾达,闰土股份,天山铝业,卫星化学,宏大爆破,奥瑞金,天赐材料,裕同科技,联泓新科,新宙邦,三聚环保,华峰超纤,国瓷材料,寒锐钴业,光威复材
工业(100):
中远海能,中直股份,四川路桥,葛洲坝,特变电工,建发股份,上海建工,平高电气,洪都航空,山东高速,中航电

,日期,原材料,信息技术,公用事业,医药卫生,能源,工业,金融地产,主要消费,电信业务,可选消费,ALL
0,2021-11-12,27.78,93.75,32.0,62.5,0.0,49.49,44.83,56.25,83.33,64.91,52.71
1,2021-11-11,27.78,87.5,24.0,60.42,7.14,47.47,48.28,65.62,83.33,64.91,52.1
2,2021-11-10,26.67,87.5,28.0,56.25,7.14,44.44,22.41,68.75,58.33,54.39,46.49
3,2021-11-09,31.11,85.94,32.0,45.83,7.14,48.48,15.52,68.75,33.33,57.89,46.09
4,2021-11-08,26.67,78.12,24.0,35.42,7.14,49.49,15.52,71.88,33.33,54.39,42.89
5,2021-11-05,20.0,76.56,12.0,39.58,7.14,46.46,6.9,81.25,33.33,59.65,40.88
6,2021-11-04,21.11,65.62,20.0,41.67,14.29,53.54,12.07,90.62,50.0,49.12,42.28
7,2021-11-03,21.11,62.5,16.0,37.5,7.14,51.52,10.34,75.0,33.33,40.35,38.08
8,2021-11-02,17.78,59.38,16.0,31.25,7.14,49.49,8.62,59.38,33.33,33.33,34.07
9,2021-11-01,20.0,57.81,24.0,31.25,7.14,47.47,12.07,56.25,25.0,40.35,35.07
